In [1]:
#Active environment should be cgcnn_matai
!conda info | grep 'cgcnn_matai'

     active environment : cgcnn_matai
    active env location : /home/hassan101/anaconda3/envs/cgcnn_matai


In [2]:
import pandas as pd
import json
import boto3
import io
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import os
MAPI_KEY = os.environ['MAPI_KEY']
aws_akid = os.environ['AWS_KID']
aws_sak = os.environ['AWS_AK']

In [4]:
#AWS authentication
res_s3 = boto3.resource('s3', aws_access_key_id=aws_akid, aws_secret_access_key= aws_sak)
client_s3 = boto3.client('s3', aws_access_key_id=aws_akid, aws_secret_access_key= aws_sak) 

In [ ]:
# List all objects in bucket
bucket_name = 'datasets-cgcnn'

response = client_s3.list_objects_v2(Bucket=bucket_name)
for obj in response['Contents']:
    print(obj)

In [55]:
def load_from_s3(file_path):
    '''
    Use this function as:
    pandas_df = d.read_excel(io.BytesIO(load_from_s3('path/to/test_file.xlsx').read())) #Sheet1 will be loaded by default
    '''
    response = client_s3.get_object(Bucket=bucket_name, Key=file_path).get("Body")
    print("FILE LOADED")
    return response

def save_to_s3(df, file_path):
    '''
    Use this function as:
    save_to_s3(pandas_df, 'path/to/test_file.xlsx')
    '''    
    with io.BytesIO() as output:
        with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
            df.to_excel(writer)
        data = output.getvalue()
    client_s3.put_object(Bucket=bucket_name, Key=file_path, Body=data)
    print('FILE SAVED')

## PROCESSING STEP -1

This step is the transformation wrapper for raw data

In [ ]:
#Query PBE band gaps from materials project to prepare DS1 for M1 development

df = pd.read_csv(
    client_s3.get_object(Bucket=bucket_name, Key="raw/mpids_for_m1.csv").get("Body"),
    header=None
    )

QUERY = list(df[0].values)
print(len(QUERY))

from pymatgen.ext.matproj import MPRester
from pymatgen.io.cif import CifParser
MAPI_KEY = MAPI_KEY
mpr = MPRester(MAPI_KEY)

prop_bg = []
from pymatgen.ext.matproj import MPRestError
for i in range(0,len(QUERY)):
    try:
        prop_bg.append(mpr.query(QUERY[i], ['material_id','pretty_formula', 'band_gap']))
    except MPRestError:
        print('Error:',QUERY[i])
    else:
       pass

# This is to check if there are any empty entries
for i in range(0,len(prop_bg)):
    if prop_bg[i] == []:
        print(str(i) + ' is empty')
        #prop_bg.remove(prop_bg[i])
        #print(str(prop_bg[i]) + ' is deleted')
       
# If there are some [] entries in prop_bg then running code below will give out of range error
import pandas as pd
df_prop_bg = pd.DataFrame(data =[])
df_prop_bg['mpids'] = [p[0]['material_id'] for p in prop_bg]
df_prop_bg['formula'] = [p[0]['pretty_formula'] for p in prop_bg]
df_prop_bg['BG_MP'] = [p[0]['band_gap'] for p in prop_bg]

df_prop_bg.to_excel('Datasets/processed_step1/Dataset_DS1_fromraw.xlsx')

In [56]:
# Preparing preliminary DS2 dataset for M2 development using data mined from literature
file = 'raw/from_lit_for_m2.xlsx'

import pandas as pd
from pymatgen.ext.matproj import MPRester
from pymatgen.io.cif import CifParser


MAPI_KEY = MAPI_KEY
mpr = MPRester(MAPI_KEY)
   
df = pd.read_excel(io.BytesIO(load_from_s3(file).read())).dropna(how='any').reset_index(drop=True)

QUERY = df['MaterialsId'].values.tolist()

print('Total entries:', len(QUERY))



prop_bg = []
for i in range(0,len(QUERY)):
    #print(mpr.get_data(QUERY[i], prop = 'band_gap')) #To check
    prop_bg.append(mpr.query(QUERY[i], ['material_id', "pretty_formula", "spacegroup.number", "band_gap"]))
    
df_prop_bg = pd.DataFrame(data =[])
df_prop_bg['mpids'] = [p[0]['material_id'] for p in prop_bg]
df_prop_bg['Formula'] = [p[0]['pretty_formula'] for p in prop_bg]
df_prop_bg['SG'] = [p[0]['spacegroup.number'] for p in prop_bg]
df_prop_bg['BG_MP'] = [p[0]['band_gap'] for p in prop_bg]
df_prop_bg['BG_exp'], df_prop_bg['Formula_exp'] = df['BG_exp'], df['Formula']

df_prop_bg.to_excel('Datasets/processed_step1/Dataset_DS2_fromrawX.xlsx')


/home/hassan101/anaconda3/envs/cgcnn_matai/lib/python3.11/site-packages/pymatgen/ext/matproj.py:182: UserWarning: You are using the legacy MPRester. This version of the MPRester will no longer be updated. To access the latest data with the new MPRester, obtain a new API key from https://materialsproject.org/api and consult the docs at https://docs.materialsproject.org/ for more information.
  warnings.warn(


FILE LOADED
Total entries: 189


In [ ]:
%%writefile get_mpids_from_formulas_sg.py
##############
# This additional script is to get mpids using formula and SG
##############
import os
from pymatgen.ext.matproj import MPRester
from pymatgen.io.cif import CifParser
MAPI_KEY = os.environ['MAPI_KEY']
mpr = MPRester(MAPI_KEY)

formulas = [
'CuGaSe2',
'CuGaSe2',
'CuInSe2',
#'CuIn0.5Ga0.5Se2',
'CdTe',
'ZnTe',
'ZnTe',
'CdSe',
'CsPbI3',
'CsGeI3',
# 'Sr15Ga22As32',
# 'Eu15Ga22As32',
# 'Sr15In22As32',
# 'Eu15In22As32',
# 'Sr3Ga6As8',
# 'Eu3Ga6As8',
'GaAs',
'InAs',
'InP',
'AlSb',
'GaP',
'GaSb'
]
mpids = []
for i in range (0,len(formulas)):
    mpids.append(mpr.get_materials_ids(formulas[i]))
    print(mpr.get_materials_ids(formulas[i]))
    
mpids_concat = []
for j in mpids:
    for i in range (0,len(j)):
        mpids_concat.append(j[i])
print(mpids_concat)

data = []
for i in range(0,len(mpids_concat)):
    data.append(mpr.query(mpids_concat[i], ['material_id', "pretty_formula", "spacegroup.number", "band_gap"])) 

import pandas as pd
df = pd.DataFrame(data = [])
df['mpids'] = [p[0]['material_id'] for p in data]
df['Formula'] = [p[0]['pretty_formula'] for p in data]
df['SG'] = [p[0]['spacegroup.number'] for p in data]
df['BG_MP'] = [p[0]['band_gap'] for p in data]

df.to_excel('Datasets/processed_step1/mpids_from_formulas_sg.xlsx')

For validation data, the dataset of experimental band gaps sourced from literature was queried against materials project to find mpids, space groups and PBE band gaps. There were multiple mpids-space group combinations for same formulas, so manual sorting was done to obtain single dataset `Dataset_for_validation.xlsx` from `Dataset_for_validation_manual_1.xlsx` and `Dataset_for_validation_manual_2.xlsx`. The manual matching was based on stability of the crystals. 

In [57]:
from pymatgen.ext.matproj import MPRester
from pymatgen.io.cif import CifParser
MAPI_KEY = MAPI_KEY
mpr = MPRester(MAPI_KEY)

file = 'raw/from_lit_for_val.xlsx'
df_raw = pd.read_excel(io.BytesIO(load_from_s3(file).read())).dropna(how='any').reset_index(drop=True)

formulas = df_raw.Formula.values

mpids = []
for i in range (0,len(formulas)):
    mpids.append(mpr.get_materials_ids(formulas[i]))
    print(mpr.get_materials_ids(formulas[i]))
    
mpids_concat = []
for j in mpids:
    for i in range (0,len(j)):
        mpids_concat.append(j[i])
print(mpids_concat)

data = []
for i in range(0,len(mpids_concat)):
    data.append(mpr.query(mpids_concat[i], ['material_id', "pretty_formula", "spacegroup.number", "band_gap"])) 

import pandas as pd
df = pd.DataFrame(data = [])
df['mpids'] = [p[0]['material_id'] for p in data]
df['Formula'] = [p[0]['pretty_formula'] for p in data]

df_raw.to_excel('Datasets/processed_step1/Dataset_for_validation_manual_1.xlsx')
df.to_excel('Datasets/processed_step1/Dataset_for_validation_manual_2.xlsx')

/home/hassan101/anaconda3/envs/cgcnn_matai/lib/python3.11/site-packages/pymatgen/ext/matproj.py:182: UserWarning: You are using the legacy MPRester. This version of the MPRester will no longer be updated. To access the latest data with the new MPRester, obtain a new API key from https://materialsproject.org/api and consult the docs at https://docs.materialsproject.org/ for more information.
  warnings.warn(


FILE LOADED
['mp-8880', 'mp-997569', 'mp-1550']
['mp-1228806', 'mp-1018100', 'mp-2624', 'mp-15621', 'mp-1023918']
['mp-984718', 'mp-10044']
['mp-422']
['mp-1008524', 'mp-11335', 'mp-1541']
['mp-252']
['mp-1008559', 'mp-1479']
['mp-3772', 'mp-33397']
['mp-672', 'mp-2469', 'mp-1181862', 'mp-1021511', 'mp-370']
['mp-2691', 'mp-1055', 'mp-1070']
['mp-3078']
['mp-406', 'mp-685146', 'mp-1492', 'mp-2388', 'mp-1066480', 'mp-1226722', 'mp-12581', 'mp-1008471', 'mp-12779']
['mp-406', 'mp-685146', 'mp-1492', 'mp-2388', 'mp-1066480', 'mp-1226722', 'mp-12581', 'mp-1008471', 'mp-12779']
['mp-1059094', 'mp-1244899', 'mp-1245274', 'mp-15619', 'mp-2534', 'mp-1232355', 'mp-10048', 'mp-1245074', 'mp-1245314', 'mp-603640', 'mp-8883', 'mp-1244999']
['mp-1245288', 'mp-1245188', 'mp-1007824', 'mp-830', 'mp-804', 'mp-1245130', 'mp-1244866', 'mp-1245046', 'mp-1245295', 'mp-1244984', 'mp-2853', 'mp-1245138', 'mp-1244975', 'mp-1245095', 'mp-1245286', 'mp-1245233', 'mp-1245172', 'mp-1245258', 'mp-1245069', 'mp-12

In [ ]:
# We got additional datasets for enrichment of DS2

from pymatgen.ext.matproj import MPRester
from pymatgen.io.cif import CifParser
MAPI_KEY = MAPI_KEY
mpr = MPRester(MAPI_KEY)


file = 'raw/from_lit_for_m2_enrichment_a.xlsx'
import pandas as pd
df_orig = pd.read_excel(io.BytesIO(load_from_s3(file).read())).dropna(how='any').reset_index(drop=True)

formulas = df_orig['Composition'].values.tolist()

print('Total entries:', len(formulas))


from pymatgen.ext.matproj import MPRestError
mpids = []
mpids_formulas = []
for i in range (0,len(formulas)):
    try:
        mpids.append(mpr.get_materials_ids(formulas[i]))
    except MPRestError:
        print('Remove this formula:', formulas[i])
    else:
         mpids_formulas.append(mpr.query(mpids[i][0], ["pretty_formula"]))
                
a = []
for i in range(0, len(mpids_formulas)):
    a.append(mpids_formulas[i][0]['pretty_formula'])
mpids_formulas = a

df_orig['Formula'] = mpids_formulas

mpids_concat = []
for j in mpids:
    for i in range (0,len(j)):
        mpids_concat.append(j[i])
print(mpids_concat)

data = []
for i in range(0,len(mpids_concat)):
    data.append(mpr.query(mpids_concat[i], ['material_id', "pretty_formula", "spacegroup.number", "band_gap"])) 

import pandas as pd
df = pd.DataFrame(data = [])
df['mpids'] = [p[0]['material_id'] for p in data]
df['Formula'] = [p[0]['pretty_formula'] for p in data]
df['SG'] = [p[0]['spacegroup.number'] for p in data]
df['BG_MP'] = [p[0]['band_gap'] for p in data]

rslt_df_1 = pd.DataFrame(data=[])
rslt_df_2 = pd.DataFrame(data=[])
for i in range(0,len(df_orig)):
    rslt_df_1 = df[(df['Formula'] == df_orig.loc[i]['Formula']) & 
                 (df['SG'] == df_orig.loc[i]['G'])
                 ]
    rslt_df_1['Eg(Exp)'] =  df_orig.loc[i]['Eg(Exp)']
    if len(rslt_df_1) > 1:
        print("Multiple entries", rslt_df_1)
        rslt_df_1 = rslt_df_1.iloc[0]
        print("First one is selected")
    rslt_df_2 = rslt_df_2.append(rslt_df_1)
    
rslt_df_2.to_excel('Datasets/processed_step1/Dataset_for_enrichment_aX.xlsx')

In [59]:
#For enrich_b, we are applying minimal changes at this step. Will process this heavily later.

file = 'raw/from_lit_for_m2_enrichment_b.xlsx'
enrich_b = pd.read_excel(io.BytesIO(load_from_s3(file).read())).reset_index(drop=True)

# Drop mp-557056, mp-24850, mp-25054, mp-25176, mp-542885
enrich_b = enrich_b.drop(enrich_b.index[enrich_b['mpids'] == 'mp-557056']).reset_index(drop=True)
enrich_b = enrich_b.drop(enrich_b.index[enrich_b['mpids'] == 'mp-24850']).reset_index(drop=True)
enrich_b = enrich_b.drop(enrich_b.index[enrich_b['mpids'] == 'mp-25054']).reset_index(drop=True)
enrich_b = enrich_b.drop(enrich_b.index[enrich_b['mpids'] == 'mp-25176']).reset_index(drop=True)
enrich_b = enrich_b.drop(enrich_b.index[enrich_b['mpids'] == 'mp-542885']).reset_index(drop=True)

text = enrich_b.columns[-1]
enrich_b = enrich_b.rename(columns = {text: 'ref'})
text = '10.1007/978-3-642-18865-7, Semiconductors: Data Handbook, Otfried Madelung, Springer-Verlag Berlin Heidelberg, 2004'
for i in range (len(enrich_b)):
    if pd.isnull(enrich_b.loc[i,'ref']):
        enrich_b.at[i,'ref'] = text

enrich_b.to_excel('Datasets/processed_step1/Dataset_for_enrichment_bX.xlsx')

FILE LOADED


## PROCESSING STEP - 2

The purpose of this step is as follows:
- Wrangling datasets based on band gap cut-off of 5 eV
- Removing validation samples from datasets used for developing M1 and M2 models

In [ ]:
bg_cutoff = 5

file_1 = 'Datasets/processed_step1/Dataset_DS1_fromraw.xlsx'
file_2 = 'Datasets/processed_step1/Dataset_DS2_fromraw.xlsx'
file_3 = 'Datasets/processed_step1/Dataset_for_validation.xlsx'
file_4 = 'Datasets/processed_step1/Dataset_for_enrichment_a.xlsx'
file_5 = 'Datasets/processed_step1/Dataset_for_enrichment_b.xlsx'


m1 = pd.ExcelFile(file_1).parse('Sheet1').rename(str.lower, axis='columns')
m2 = pd.ExcelFile(file_2).parse('Sheet1').rename(str.lower, axis='columns')
validation = pd.ExcelFile(file_3).parse('Sheet1').rename(str.lower, axis='columns')
enrich_a = pd.ExcelFile(file_4).parse('Sheet1').rename(str.lower, axis='columns')
enrich_b = pd.ExcelFile(file_5).parse('Sheet1').reset_index(drop=True)

In [ ]:
# Processing M1
processed_m1 = m1[(m1['bg_mp'] < bg_cutoff) ].drop('unnamed: 0', 1).reset_index(drop=True)

# Processing validation
processed_validation = validation.rename(columns={'exp.':'bg_exp',
                                                  'hse':'bg_hse',
                                                  'g0w0':'bg_gw'
                                                  }).drop('unnamed: 0', 1).reset_index(drop=True)

# Removing val from M1
overlap_m1_val = processed_m1[processed_m1.mpids.isin(processed_validation.mpids)]
processed_m1_noval = processed_m1[~processed_m1.mpids.isin(processed_validation.mpids)]

# Processing M2
m2.drop('formula_exp', 1, inplace = True)

for i in range(0,len(m2)):
    if type(m2.iloc[i]['bg_exp']) is str:
        print ('index:',i,'\n',m2.iloc[i], '\n')
m2.at[98, 'bg_exp'] , m2.at[99, 'bg_exp'] = 3.8 , 3.7
m2['bg_exp'] = pd.to_numeric(m2['bg_exp'],errors = 'coerce')

m2_norep = m2.drop_duplicates(subset=['mpids'], keep='first')

processed_m2 = m2_norep.drop('unnamed: 0', 1).reset_index(drop=True)

overlap_m2_val = processed_m2[processed_m2.mpids.isin(processed_validation.mpids)]
processed_m2_noval = processed_m2[~processed_m2.mpids.isin(processed_validation.mpids)].reset_index(drop=True)

processed_m2 = processed_m2[(processed_m2['bg_exp'] < bg_cutoff) ].reset_index(drop=True)
processed_m2_noval = processed_m2_noval[(processed_m2_noval['bg_exp'] < bg_cutoff) ].reset_index(drop=True)

# Processing enrichment dataset & Concat with M2
enrich_a.rename(columns={'eg(exp)':'bg_exp'}, inplace = True)

m2_enrich_a = pd.concat([m2_norep, enrich_a], axis = 0)
m2_enrich_a.drop_duplicates(subset=['mpids'], keep='first', inplace = True)

processed_m2_enrich_a = m2_enrich_a.drop('unnamed: 0', 1).reset_index(drop=True)

overlap_m2_enrich_a_val = processed_m2_enrich_a[processed_m2_enrich_a.mpids.isin(processed_validation.mpids)]
processed_m2_enrich_a_noval = processed_m2_enrich_a[~processed_m2_enrich_a.mpids.isin(processed_validation.mpids)].reset_index(drop=True)

processed_m2_enrich_a = processed_m2_enrich_a[(processed_m2_enrich_a['bg_exp'] < bg_cutoff) ].reset_index(drop=True)
processed_m2_enrich_a_noval = processed_m2_enrich_a_noval[(processed_m2_enrich_a_noval['bg_exp'] < bg_cutoff) ].reset_index(drop=True)

In [ ]:
#Save files
processed_m1_noval.to_excel('Datasets/processed_step2/processed_ds1.xlsx')
processed_m2_enrich_a_noval.to_excel('Datasets/processed_step2/processed_ds2_a.xlsx')
processed_validation.to_excel('Datasets/processed_step2/processed_ds3.xlsx')

In [ ]:
# Plot distributions of bandgaps in validation set
import seaborn as sns
import matplotlib.pyplot as plt

ax = sns.displot(processed_validation, x="bg_mp", kde=True)
ax.set(xlabel='DFT(PBE) bandgaps', ylabel='Count')
plt.tight_layout()
plt.show()

Preparing enrichment set b for DS2

In [ ]:
from pymatgen.ext.matproj import MPRester
from pymatgen.io.cif import CifParser
MAPI_KEY = MAPI_KEY
mpr = MPRester(MAPI_KEY)

import pandas as pd

enrich_b = enrich_b.drop('Unnamed: 0', 1).reset_index(drop=True)

#Drop duplicates
# Remove ds2_a from enrich_b
enrich_b = enrich_b[~enrich_b.mpids.isin(processed_m2_enrich_a_noval.mpids)].reset_index(drop=True)

# Remove validation from enrich_b
enrich_b = enrich_b[~enrich_b.mpids.isin(processed_validation.mpids)].reset_index(drop=True)

# Check overlap
print(enrich_b[enrich_b.mpids.isin(processed_m2_enrich_a_noval.mpids)])
print(enrich_b[enrich_b.mpids.isin(processed_validation.mpids)])
print(enrich_b[enrich_b.mpids.isin(processed_m1_noval.mpids)]) #Overlap with DS1 does not matter since all of these are going to DS2 for training. The print for above two must return an empty list.    

In [ ]:
#Get bandgaps from Materials Project

QUERY = enrich_b['mpids'].values.tolist()

prop_bg = []
from pymatgen.ext.matproj import MPRestError
for i in range(0,len(QUERY)):
    try:
        prop_bg.append(mpr.query(QUERY[i], ['band_gap']))
    except MPRestError:
        print('MPRestError for:',QUERY[i])
    else:
       pass

In [ ]:
# This is to check if there are any empty entries
for i in range(0,len(prop_bg)):
    if prop_bg[i] == []:
        print(str(i) + ' is empty')
        #prop_bg.remove(prop_bg[i])
        #print(str(prop_bg[i]) + ' is deleted')
       
# If there are some [] entries in prop_bg then running code below will give out of range error
enrich_b['bg_mp'] = [p[0]['band_gap'] for p in prop_bg]

In [ ]:
##Check structure and get cifs
# structures_cif = []
# for i in range(0,len(QUERY)):
#     structures_cif.append(mpr.get_data(QUERY[i], data_type = '', prop = 'cif').get('cif'))
# print("No. of structures:",len(structures_cif))

# #Save structures in cif file
# for i in range(0,len(structures_cif)):
#     with open('cif/{}.cif'.format(QUERY[i]),'w') as output:
#         output.write(structures_cif[i])

In [ ]:
# Remove items outside bg threshold
bg_cutoff = 5
enrich_b = enrich_b[ (enrich_b['bg_exp'] <= bg_cutoff)].reset_index(drop=True)

#Rearrange columns
cols = enrich_b.columns.tolist()
cols_new = cols[:-2] + cols[-1:] + cols[-2:-1]
enrich_b = enrich_b[cols_new]

enrich_b.to_excel('Datasets/processed_step2/processed_ds2_b.xlsx')

## FINAL PROCESSING STEP

Concatenating DS2 subsets and saving all finalized datasets

In [60]:
df_ds1 = pd.ExcelFile('Datasets/processed_step2/processed_ds1.xlsx').parse('Sheet1').rename(str.lower, axis='columns')
df_ds2_a = pd.ExcelFile('Datasets/processed_step2/processed_ds2_a.xlsx').parse('Sheet1').rename(str.lower, axis='columns')
df_ds2_b = pd.ExcelFile('Datasets/processed_step2/processed_ds2_b.xlsx').parse('Sheet1').rename(str.lower, axis='columns')
df_ds3 = pd.ExcelFile('Datasets/processed_step2/processed_ds3.xlsx').parse('Sheet1').rename(str.lower, axis='columns')

df_ds2 = pd.concat( [ df_ds2_a[['mpids','formula','bg_mp','bg_exp']] , df_ds2_b[['mpids','formula','bg_mp','bg_exp']]  ] , axis = 0 ).reset_index(drop=True)

df_ds1 = df_ds1.drop('unnamed: 0',1)
df_ds3 = df_ds3.drop('unnamed: 0',1)

#Saving finalized spread sheets to AWS S3 bucket
save_to_s3(df_ds1, 'processed/processed_ds1.xlsx')
save_to_s3(df_ds2, 'processed/processed_ds2.xlsx')
save_to_s3(df_ds3, 'processed/processed_ds3.xlsx')

/tmp/ipykernel_24727/178158609.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_ds1 = df_ds1.drop('unnamed: 0',1)
/tmp/ipykernel_24727/178158609.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_ds3 = df_ds3.drop('unnamed: 0',1)


FILE SAVED
FILE SAVED
FILE SAVED


In [ ]:
#Saving JSON
df_ds1 = df_ds1.set_index('mpids')
df_ds1_json = df_ds1.to_json(orient="index")
df_ds1_json = json.loads(df_ds1_json)

df_ds2 = df_ds2.set_index('mpids')
df_ds2_json = df_ds2.to_json(orient="index")
df_ds2_json = json.loads(df_ds2_json)

df_ds3 = df_ds3.set_index('mpids')
df_ds3_json = df_ds3.to_json(orient="index")
df_ds3 = json.loads(df_ds3_json)

with open('Datasets/json/ds1_json.json', 'w') as f:
    json.dump(df_ds1_json, f,indent=2)
    
with open('Datasets/json/ds2_json.json', 'w') as f:
    json.dump(df_ds2_json, f,indent=2) 
    
with open('Datasets/json/ds3_json.json', 'w') as f:
    json.dump(df_ds3_json, f,indent=2)